Match the APOGEE-red-clump catalog to Gaia DR2, integrate the orbits using galpy for 20 Gyr, and make a density plot in Galactocentric X and Y coordinates of all of the positions of the stars over the next 20 Gyr.

In [ ]:
#import os
# set environment variable for gaia_tools.load to work
#os.environ['GAIA_TOOLS_DATA'] = "C:/Users/user/Git Hub Projects/GAIA_TOOLS_DATA"

In [32]:
import astropy.units as u
import gaia_tools.load as gload
import gaia_tools.xmatch as gmatch
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
from astropy.coordinates import SkyCoord
import numpy as np

load the data from APOGEE red clump

In [24]:
aprc= gload.apogeerc()
gaia2_matches, matches_indx = gmatch.cds(aprc, xcat='vizier:I/345/gaia2')

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\gaia_tools-0.2-py3.6.egg\gaia_tools\load\__init__.py:94: UserWarning: Falling back on simple APOGEE interface; for more functionality, install the jobovy/apogee package
  warnings.warn("Falling back on simple APOGEE interface; for more functionality, install the jobovy/apogee package")


In [25]:
gaia2_matches.dtype

dtype([('angDist', '<f8'), ('RA', '<f8'), ('DEC', '<f8'), ('ra_epoch2000', '<f8'), ('dec_epoch2000', '<f8'), ('errHalfMaj', '<f8'), ('errHalfMin', '<f8'), ('errPosAng', '<f8'), ('source_id', '<f8'), ('ra', '<f8'), ('ra_error', '<f8'), ('dec', '<f8'), ('dec_error', '<f8'), ('parallax', '<f8'), ('parallax_error', '<f8'), ('pmra', '<f8'), ('pmra_error', '<f8'), ('pmdec', '<f8'), ('pmdec_error', '<f8'), ('duplicated_source', '<f8'), ('phot_g_mean_flux', '<f8'), ('phot_g_mean_flux_error', '<f8'), ('phot_g_mean_mag', '<f8'), ('phot_bp_mean_flux', '<f8'), ('phot_bp_mean_flux_error', '<f8'), ('phot_bp_mean_mag', '<f8'), ('phot_rp_mean_flux', '<f8'), ('phot_rp_mean_flux_error', '<f8'), ('phot_rp_mean_mag', '<f8'), ('bp_rp', '<f8'), ('radial_velocity', '<f8'), ('radial_velocity_error', '<f8'), ('teff_val', '<f8'), ('a_g_val', '<f8'), ('e_bp_min_rp_val', '<f8'), ('radius_val', '<f8'), ('lum_val', '<f8')])

In [30]:
ra_list = gaia2_matches['ra']
dec_list = gaia2_matches['dec']
parallax_list = gaia2_matches['parallax']
radial_velocity_list = gaia2_matches['radial_velocity']
proper_motion_ra = gaia2_matches['pmra']
proper_motion_dec = gaia2_matches['pmdec']

19282

19282